In [1]:
# import de modulos pandas e numpy
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

# Math
import math

# import de modulos para graficos
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# imports para Machine Learning 
from sklearn.linear_model import LogisticRegression
# from sklearn.cross_validation import train_test_split # modulo antigo
from sklearn.model_selection import train_test_split

# para avaliacao
from sklearn import metrics

# dataset
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Função Logistica
def logistic(t):
    return 1.0 / (1 + math.exp((-1.0)*t) )

# cria um grade de -6 a 6 ( 500 elementos, igualmente espaçados)
grade_t = np.linspace(-6,6,500)

# calcula os valores de y
p = np.array([logistic(t) for t in grade_t])

# usando exp do numpy que aceita um vetor como argumento
p = 1/(1 + np.exp(-1.0 * grade_t))

# plot
plt.plot(grade_t,p)
plt.title(' Funcao Logistica ');

In [ ]:
import webbrowser

url = 'http://statsmodels.sourceforge.net/stable/datasets/generated/fair.html'

webbrowser.open_new(url)

In [ ]:
df = sm.datasets.fair.load_pandas().data

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
# Define uma coluna, indicador binário que indica se a pessoa teve um caso extra-conjugal
df['teve_affair'] = [1 if a else 0 for a in df.affairs]

In [ ]:
df.sample(5)

In [ ]:
# Media por cada label
df.groupby('teve_affair').mean().round(2)

In [ ]:
# desvio padrão por cada label
df.groupby('teve_affair').std()

In [ ]:
sns.set_context('poster')

In [ ]:
df.affairs.describe(percentiles=[0, 0.9, 0.99])

In [ ]:
# Countplot com idade
sns.countplot(data=df, x='age', hue='teve_affair', palette='coolwarm')
# sns.distplot(df[df.teve_affair == 0].age)
# sns.distplot(df[df.teve_affair == 1].age)

In [ ]:
# Countplot com anos de casamento
sns.countplot('yrs_married',data=df,hue='teve_affair',palette='coolwarm')

In [ ]:
# Countplot com quantidade de filhos
sns.countplot('children',data=df,hue='teve_affair',palette='coolwarm')

In [ ]:
# Countplot com nivel de educação
sns.countplot('educ',data=df,hue='teve_affair',palette='coolwarm')

In [ ]:
# considerando que a pessoa teve um caso extra-conjugal, 
# visualizacão da distribuição por boxplot
sns.boxplot(data=df[df.affairs > 0], x='age', y = 'affairs')
plt.ylim(0, 15)

In [ ]:
# considerando que a pessoa teve um caso extra-conjugal, 
# visualizacão da distribuição por violinplot
sns.violinplot(data=df[(df.affairs > 0) & (df.affairs < 9)], x='age', y = 'affairs')
plt.ylim(-2, 12)

In [ ]:
sns.pairplot(data=df[['affairs', 'age', 'yrs_married', 'children', 'religious']], 
        plot_kws = {'alpha':0.08, 's':180, 'edgecolor': None},)

In [ ]:
df.sample(5)

In [ ]:
# Cria novo DataFrame para as colunas categoricas
occ_dummies = pd.get_dummies(df['occupation'])
hus_occ_dummies = pd.get_dummies(df['occupation_husb'])

occ_dummies.head()

In [ ]:
# Atribui nomes as colunas
occ_dummies.columns = ['occ1','occ2','occ3','occ4','occ5','occ6']
hus_occ_dummies.columns = ['hocc1','hocc2','hocc3','hocc4','hocc5','hocc6']

In [ ]:
# Atribui X primeiramente sem as colunas categoricas 
X = df.drop(['occupation','occupation_husb','teve_affair'],axis=1)

# Concatena os dataframes dummies
dummies = pd.concat([occ_dummies,hus_occ_dummies],axis=1)

In [ ]:
# Concatena o X com o dataframe dos dummies
X = pd.concat([X,dummies],axis=1)

# amostragem do X
X.sample(5)

In [ ]:
# deletando uma coluna para cada coluna categorica
X = X.drop('occ1',axis=1)
X = X.drop('hocc1',axis=1)

# deletando coluna não utilizada
X = X.drop('affairs',axis=1)

# amostragem
X.sample(5)

In [ ]:
# Atribui y a coluna teve_affair
y = df.teve_affair

# amostragem do y
y.head()

In [ ]:
# transformando em numpy
y = y.values

# checando resultado
y

In [ ]:
y.mean()

In [ ]:
# Instanciando objeto
log_model = LogisticRegression()

# Treinando o modelo
log_model.fit(X, y)

# Checando acurácia
log_model.score(X, y)

In [ ]:
log_model

In [ ]:
y.mean()

In [ ]:
coeff_df = DataFrame(log_model.coef_)

coeff_df.columns = X.columns

coeff_df = coeff_df.T

coeff_df.columns = ['coeficiente']

In [ ]:
coeff_df.plot(kind='bar', figsize=(12,6))

In [ ]:
from sklearn import metrics

In [ ]:
metrics.confusion_matrix(y, log_model.predict(X))

In [ ]:
# Segmentando a base
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)

# Instanciando um objeto
log_model2 = LogisticRegression()

# Treinando o modelo
log_model2.fit(X_train, y_train)

In [ ]:
# Predizendo a classe das observações de teste
class_predict = log_model2.predict(X_test)

# Comparando as classes da predição e o gold, 
# ou seja, medindo a performance...
print (metrics.accuracy_score(y_test,class_predict))

In [ ]:
metrics.confusion_matrix(y_test, log_model2.predict(X_test))

In [ ]:
print(metrics.classification_report(y_test, log_model2.predict(X_test)))

In [ ]:
log_model2.coef_

In [ ]:
log_model2.intercept_

In [ ]:
X_train.columns

In [ ]:
y_test

In [ ]:
class_predict

In [ ]:
(log_model2.predict_proba(X_test)[:,1] > 0.2).astype(int)

In [ ]:
# matriz de confusão
confusion_matrix = metrics.confusion_matrix(y_test, class_predict)
confusion_matrix

In [ ]:
# Label gold na vertical, e label predição na horizontal
sns.heatmap(confusion_matrix, annot=True, fmt ='d')

In [ ]:
print(metrics.classification_report(y_test, class_predict))

In [ ]:
confusion_matrix/confusion_matrix.sum()

In [ ]:
y_pred = log_model2.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, y_pred)